# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

`06_data_summary.ipynb`

Requires:

1. Cleaned trade data created by `02_clean.ipynb`
2. `product_code_baci_07.csv` from CEPII: http://www.cepii.fr/DATA_DOWNLOAD/baci/country_code_baci07.csv
3. Distribution estimates from `03_deg_dist.ipynb`

Creates a csv file for each year with the list of product codes, the value and volume of trade, the number of imports and number of exporters, the alpha value estimated in `03_deg_dist`, the standard deviation of the estimate, and the full name of the product. 

---

METODO:

1. Speed up run time if poss.

## Product Level Summary

#### Packages

In [1]:
import pandas as pd                                 # pandas dataframes used for convenience
import os 

os.chdir('C:/Working/trade_network/data/')          # change current directory in next line

#### Basic summary: Ranking of Products


In [2]:
# Create a dictionary of HS six digit level product codes 
prod_names = pd.read_csv('product_code_baci07.csv', index_col='hs6')['name']

for y in map(str, range(2008,2015)):
    baci_clean = 'clean/baci07_'+y+'_clean.csv'        # the cleaned dataset for each year
    alpha_csv = 'summary/deg_dist_alpha_'+y+'.csv'     # alpha values estimated previously
    # Calcuate value and quantity/volume:
    val_qty_frame = pd.read_csv(baci_clean)[['hs6', 'v', 'q']].groupby('hs6').sum()
    # Calculate number of exporters:
    num_x = pd.read_csv(baci_clean)[['hs6', 'i']].groupby('hs6')['i'].nunique()
    # Calculate number of importers:
    num_m = pd.read_csv(baci_clean)[['hs6', 'j']].groupby('hs6')['j'].nunique()
    # Open alpha scores, merge them in and save results as CSV:
    alpha = pd.read_csv(alpha_csv, header=None).rename(columns={0: 'hs6', 1: 'alpha', 2: 'sigma'})
    full_year = val_qty_frame.join(num_x).join(num_m).join(alpha.set_index('hs6'))
    full_year.join(prod_names).to_csv('summary/product_'+y+'.csv')